In [1]:
import sklearn.metrics as metrics
import numpy as np
import scipy.io as sio
import math
import csv
import matplotlib.pyplot as plt


def load_dataset():
    datamat=sio.loadmat('./spam_data/spam_data.mat')
    X_train=datamat['training_data']
    X_test=datamat['test_data']
    y_train=datamat['training_labels']
    return X_train, X_test, y_train



In [97]:
X_train, X_test, y_train =load_dataset()

In [98]:
y_train=y_train.reshape((y_train.shape[1],1))
X_y=np.hstack((X_train,y_train))
X_y,X_validation =X_y[:-1000,:],X_y[X_train.shape[0]-1000:,:]

# Choosing attribute and a threshold for that attribute



We want to go through each feature, that is the vector of the values of the same attribute for each sample, and then span that feature with different threshold values for its attributes. Then we want to check their information gains and choose the one that gives us the maximum information gain.

The Entropy is defined as $H(\rho)=-\sum_i^{n_{class}}\rho_i\log_{2}(\rho_i)$


The expected entropy for choosing one attribute from the feature vector is defined as $EH(\rho_i)=\sum_{i}^{k}\frac{p_i+ n_i}{n + p}H(\rho_i)$


Here $p_i$ and $n_i$ represent the number of samples of the two classes in each children respectively. $n + p$ is the total number of examples of the training set.

In [20]:
#The entropy for the root
def H(X):
    if X.shape[0]==0:
        return 0
    #Takes the classes 
    labels=X[:,-1]
    examples = float(labels.shape[0])
    n=float(np.count_nonzero(labels)) #number of samples that belong to class (Y=1)
    if n==examples:
        p=0.0
    else:
        p=float(labels.shape[0]-n) #number of samples that belong to class (Y=0)
    p1=n/examples
    p2=p/examples
    log2 = lambda x: math.log(x)/math.log(2)
    return -p1*log2(p1+ 0.0001)-p2*log2(p2+0.0001)
    


In the case for only two classes, we have a binary decision tree, that means that after the feature passes through a node the decision rule of node will divide the samples in two groups. We then just need to worry about two children coming out of each parent every time.



We will then need to choose a value for our threshold and an attribute. We have multiple features and each feature is going to spand different values. To chose the best threshold using the greedy algorightm we can simply span a set of thresholds for each feature and  choose which  will be the best threshold and attribute  by checking the Information gain of each combination.

We described the information gain for a certain attribute as follows. $I(\rho_i)=H(\rho_i)-EH(\rho_i)$

In [5]:
#The last column of the rows will be its class labels for each feature

def split_set(rows,col, value):
    Lset=[]
    Rset=[]
    for row in rows:
        if row[col] >= value:
            Lset.append(row)
        else:
            Rset.append(row)
    Lset=np.array(Lset)
    Rset=np.array(Rset)
    return Lset, Rset
    
    
    

In [6]:
#Expectation Entropy function, tells us the weigthed sum  of the two children

def EH(Left,Right):
    examplesL=float(Left.shape[0]) # examples in the child on the left
    examplesR=float(Right.shape[0]) # examples in the child on the right
    p=examplesL/(examplesL + examplesR)
    return (p)*H(Left)+(1-p)*H(Right)
    

In [203]:
#Function to choose the thresholds
'''1. Sort the feature vector
   2. Starting from the smallest number compare adjecent rumbers as you ascend to higher values
   3. Only for those adjecent numbers that differ calculate the midpoint between them
   4. Return the array all the allowed midpoints'''
def thresholds(rows,column):
    feature=np.sort(np.unique(rows[:,column]))
    place =feature[0] #Lessser value
    midpoints=[]
    for i in range(feature.shape[0]-1):
        if place != feature[i+1]:
            midpoints.append((feature[i+1]+place)/2)
            place=feature[i+1]
        else:
            place=feature[i+1]
    return np.array(midpoints)
    
def thresholds(rows,column):
    return np.unique(rows[:,column])

In [8]:
# This is a test for our chooser for our decision rule for a node. That is choosing the  best feature and threshold that will
#give us the best Information gain
def infGain(entropy,expectedE):
    return entropy - expectedE

def decide_rule(X):
    #Returns an array containing the Infomation Gain, 
    results=[] 
    #This is the entropy of the parent
    entropy=H(X)
    Gain1=0.0
    thresh=0.0
    index=0
    for i in range(X.shape[1]-1):
        #This for loop iterates through all the features in the set
        Thresholds=thresholds(X,i)
        for threshold in Thresholds:
            #This for loop iterates through the selected thresholds of the ith feature
            left, right = split_set(X,i,threshold)
            #This is the Information gain 
            Gain2=infGain(entropy,EH(left,right))
            if Gain1 < Gain2:
                Gain1=Gain2
                thresh=threshold
                index=i
    return Gain1, thresh, index
    
        

# Creating a decision Tree

So far we've only been working on chooing the best decision rule of a node of a decision tree. We want to apply this recursively to all the nodes so as to get the least entropy as the last level of children. We want each leaf of the tree to be as pure as possible at the bottom, so the optimum leaf would only contain examples belonging to one single class.

So then how do build this decision tree with what we know?
Well want to essentially proceed with the same steps as we did before but span this procedure to all the necessary branches needed to come as close as we can to the optimum leaves without overfitting the tree.

We essentially wasn to apply the previous steps in a recurive matter down the decision tree. Let's recap and create a couple guidelines first.

A decision tree will take a set of examples and by applying a set of decision rules at the nodes that will split the examples in two groups (for non-binary decision tree it can be more than 2 groups) which will arrive to different nodes where another decision rule will be applied at each node . This process will continue until we get until the leaves which in theory should give us sets of pure examples in each leaf.

1.- First we can account for the impurity of the initial set of examples by calculating the entropy as shown above.

2.- Choosing the first decision rule will rely on choosing the attribute and threshold value that give the best Infomation gain(the difference between the current entropy and the expected entropy). That is this process will try to divide the whole set in two groups and  get their weighted sum of entropies or expected entropy and choose the decision rule based on which set of attribute and threshold value give the lest expected entropy .

3.- Then we know that the purpose of this algorithm is to calculate the information gain for every attribute and choosing the one that gives the highest infomation gain. We will simply do this again and again.


Then we can write this algorightm recursively following the creteria above. In my case im going to choose to implement this tree as a class for organization purposese.


In [200]:
class decisionNode:
    '''This represents the class for the nodes where the decision rule will be inserted. That is the atribute or column
    and  the threshold of value selected'''
    def __init__(self,col=-1,value=None,results=None,Lb=None,Rb=None):
        self.col=col
        self.value=value
        self.results=results
        self.Lb=Lb #left brach
        self.Rb=Rb #right branch
        
    def myclass(self):
        if self.results is not None:
            counts=np.bincount(self.results.astype(int))
            return np.argmax(counts)
    
def pure(Set):
    classes=np.unique(Set[:,Set.shape[1]-1]).shape[0]
    if classes==1:
        return True
    else:
        return False
#Creating the nodes recursively as stated before
#Main things to check is that if the examples at the certain node 
def decisionTree(Set,max_depth, count):
    if type(Set).__name__ != 'ndarray':
        raise NameError("Oops, not a ndarray. Try again.")
    if Set.shape[0]==0:
        #returns one or zero when the there are no examples on the node
        print("ho")
        tree[count][count+1]=decisionNode(results=np.array([np.random.randint(2)]))
        return tree
    y_index=Set.shape[1]-1
        
    tree ={count:{}}
    if pure(Set):
        tree[count][count+1]=decisionNode(results=Set[:,y_index])
        return tree
    #Criteria for tree
    #if currentGain==None:
        #previousGain=0.0
    #else:
        #previousGain=currentGain
    max_count=max_depth
    #Decide_rule returns the best highest gain, the threshold, and the column of the feature selected.   
    Gain, threshold, column=decide_rule(Set)
    #splits the incoming set into branches
    leftBranch,rightBranch = split_set(Set,column,threshold)
    #Creating a tree for the dictionaries
    #Spreading the branches down the tree
    if Gain>1e-6 and count<=max_count:
        leftBranch=decisionTree(leftBranch,max_depth,count=count+1)
        rightBranch=decisionTree(rightBranch,max_depth,count=count+1)
        tree[count][count+1]=decisionNode(col=column,value=threshold,Lb=leftBranch,Rb=rightBranch)
    else:
        tree[count][count+1]=decisionNode(results=Set[:,y_index])
        
    return tree
    

In [99]:
tree=decisionTree(X_y,50,1)



In [100]:
classification_accuracy(tree,X_y)

0.87392138063279

Before we can test any new data we must first realize that we have only created a decision tree and we still have figure out how to classify correctly new inconming examples. How are we going to do this well the easiest way to achieve this is by first looking at a single example and figure out  which leaf it will land on. After it may come to your attention that we have already classified training examples before so looking at those already classified examples in each node we can look for the distribution of examples in each leaf. That is if there's only one class of examples in the class then the probability that an example that lands on that leaf being of that certain class is completely 100%. If there exist a probability dirstribution for the classes then we pick the highest probability and we assign the test examples we the class of the highest probability.

In [12]:
#Classifier will essentially lead the test examples down the tree by essentially picking the attribute column and using
#the threshold stored in the decision node to apply the same procedure down the tree until we get to the bottom
def single_prediction(tree,example,count=1):
    
    node=tree[count][count+1]
  
    if node.results is not None:
        return node.myclass()
    feature_index=node.col
    threshold=node.value
    if example[feature_index]>=threshold:
        return single_prediction(node.Lb,example,count+1)
    else:
        return single_prediction(node.Rb,example,count+1)


def classification_accuracy(tree,testSet):
    "Returns the accuracy of the test"
    label_index=testSet.shape[1]-1
    correct_labels=0
    for example in testSet:
        prediction_label=single_prediction(tree,example)
        if prediction_label==int(example[label_index]):
            correct_labels+=1
    return float(correct_labels)/testSet.shape[0]
    
   
        

In [187]:

# Making a random subset of data for a tree with a set number of examples
def sampleSet(Set,size):
    subset=[]
    for i in np.arange(size):
        row=np.random.randint(Set.shape[0])
        subset.append(Set[row,:])
    return np.array(subset)
# Here we will create a Random forest tree

def randomForest(Set,subset_size,max_depth,n_trees):
    trees=[]
    for i in np.arange(n_trees):
        subset=sampleSet(Set,subset_size)
        tree=decisionTree(subset,max_depth,1)
        trees.append(tree)
    return trees
#Calculates the prediction of one example of random decision trees utilizing an extension of bagging for Random Forests
def prediction_bagging(trees,testExample):
    single_predictions=[single_prediction(tree,testExample) for tree in trees]
    counts=np.bincount(np.array(single_predictions))
    return np.argmax(counts)

def ranForest_predictions(trees,testSet):
    predictions=[]
    for row in np.arange(testSet.shape[0]):
        predictions.append(prediction_bagging(trees,testSet[row,:]))
    return np.array(predictions)



def forest_accuracy(trees,Set):
    labels=Set[:,-1]
    correct_labels=0
    predictions=ranForest_predictions(trees,Set)
    for i in np.arange(labels.shape[0]):
        if predictions[i]==int(labels[i]):
            correct_labels+=1
    return float(correct_labels)/labels.shape[0]
# Prediction for kaggle   
def predict_kaggle(trees,Set):
    predictions=ranForest_predictions(trees,Set)
    with open('kaggle2.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile)
        spamwriter.writerow(['Id','Category'])
        k=1
        for i in predictions:
            spamwriter.writerow([k, i])
            k+=1
    

In [207]:
trees=randomForest(X_y,900,16,43)


In [208]:
print ("This is the training accuracy:")
print(forest_accuracy(trees,X_y))

print ("This is the validation accuracy")
print(forest_accuracy(trees,X_validation))


This is the training accuracy:
0.832454458293
This is the validation accuracy
0.857


In [188]:
predict_kaggle(trees,X_test)